# Data Collection - Data Sources Exploration

This is a rough notebook to get started in identifying, downloading data sources.

The market data on the current shopping list is:

Sovereigns
-------------------
- US treasury note yields [wip]
- UK gilts [wip]
- DE bonds
- FR bonds

Inflation
-------------------
- EU CPI [wip]
- UK CPI [wip]
- US CPI  [wip]

Interest Rates
-------------------
- ECB [wip]
- BOE [wip]

FX Rates
-------------------
- EUR/GBP [wip]
- USD/EUR [wip]

Equity Indices
-------------------
- FTSE100 [wip]
- NASDAQ [wip]
- DOW [wip]
- S&P500 [wip]

Commodities 
-------------------
- Brent [wip]
- Grain [wip]
- Aluminium [wip]


Known data sources
-------------------
https://www.alphavantage.co/
https://polygon.io/stocks
https://data.nasdaq.com/
https://data.nasdaq.com/data/EUREX-eurex-futures-data

Packages
--------------------
quandl (https://data.nasdaq.com/tools/python)

In [11]:
import numpy as np
import pandas as pd
import quandl

In [14]:
# Insert your private Nasdaq API here
quandlapi = ""

In [18]:
def dldata (data,**kwargs):
    """
    Universal data download function that handles the different API calls based
    on what is being requested. Will ask for your identifiers where required.
    The different data sources are:
    - a
    - b
    - c
    Returns a time series
    """
    
    return data


In [46]:
def dldocu (source=""):
    """
    Downloads the available documentation and/or tickers to provide for the API
    calls needed by dldata where these are available.
    - a
    - b
    - c
    Returns a file with the tickers or display the list of options
    Leave Source empty for list of available options
    """
    available = ["EUREX Futures"]
    
    if source == "":
        print("Documentation available on the following:")
        print(available)
        return
    elif source == "EUREX Futures":
        docu = pd.read_csv("https://static.quandl.com/Ticker+CSV%27s/Futures/EUREX.csv",encoding='latin-1')
    else:
        raise ValueError("The documentation requested is not valid, check what is available")

    return docu

In [47]:
dldocu("EUREX Futures")

,Symbol,Exchange,Name,Months,Quandl Code
0,CONF,EUREX,CONF,HMUZ,EUREX/CONF
1,F2CR,EUREX,CER Emissions Mid December,Z,EUREX/F2CR
2,F2MX,EUREX,MDAX,HMUZ,EUREX/F2MX
3,FBTP,EUREX,Long-Term Euro-BTP,HMUZ,EUREX/FBTP
4,FBTS,EUREX,Short-Term Euro-BTP,HMUZ,EUREX/FBTS
...,...,...,...,...,...
66,FSMP,EUREX,Skimmed Milk Powder (EUREX),FGHJKMNQUVXZ,EUREX/FSMP
67,FSTX,EUREX,STOXX Europe 50 Index,HMUZ,EUREX/FSTX
68,FTDX,EUREX,DivDAX,HMUZ,EUREX/FTDX
69,FVS,EUREX,EURO STOXX 50 Volatility (VSTOXX),HMUZ,EUREX/FVS
